<a href="https://colab.research.google.com/github/Nam-gu/MLB_Prediction/blob/main/StartingPitcher/Starting_pitcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Statscast Stats

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm
from sklearn.metrics import accuracy_score
import lightgbm
from lightgbm import LGBMClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd drive/MyDrive/MLP_Prediction/MLB_Prediction/

/content/drive/MyDrive/MLP_Prediction/MLB_Prediction


In [4]:
file = './data/Retrosheet_2010_2019/2010_to_2019_seasons.csv'
game_df = pd.read_csv(file)

game_col_del = '''VisitorGDP,VisitorCI,HomeGDP,HomeCI, DoubleHeader, DayOfWeek, VisitingTeamLeague, HomeTeamLeague, DayNight, CompletionInfo,ForfeitInfo, ProtestInfo,Attendence, Duration, VisitorLineScore, HomeLineScore, UmpireHID, UmpireHName, Umpire1BID, Umpire1BName, Umpire2BID,Umpire2BName, Umpire3BID, Umpire3BName, UmpireLFID, UmpireLFName, UmpireRFID,UmpireRFName, VisitorManagerID, VisitorManagerName, HomeManagerID, HomeManagerName,WinningPitcherID, WinningPitcherName, LosingPitcherID, LosingPitcherNAme,SavingPitcherID, SavingPitcherName, GameWinningRBIID, GameWinningRBIName, VisitorBatting1Position, VisitorBatting2Position, VisitorBatting3Position,VisitorBatting4Position, VisitorBatting5Position, VisitorBatting6Position,VisitorBatting7Position, VisitorBatting8Position, VisitorBatting9Position,HomeBatting1Position, HomeBatting2Position, HomeBatting3Position,HomeBatting4Position, HomeBatting5Position, HomeBatting6Position,HomeBatting7Position, HomeBatting8Position, HomeBatting9Position,AdditionalInfo, AcquisitionInfo'''
game_col_del = game_col_del.replace(" ", "")
game_col_del = game_col_del.split(",")

game_df.drop(game_col_del,axis=1,inplace=True)


col_del = ['VisitorBatting1PlayerID', 'VisitorBatting1Name', 'VisitorBatting2PlayerID', 'VisitorBatting2Name', 'VisitorBatting3PlayerID', 'VisitorBatting3Name', 'VisitorBatting4PlayerID', 'VisitorBatting4Name', 'VisitorBatting5PlayerID', 'VisitorBatting5Name', 'VisitorBatting6PlayerID', 'VisitorBatting6Name', 'VisitorBatting7PlayerID', 'VisitorBatting7Name', 'VisitorBatting8PlayerID', 'VisitorBatting8Name', 'VisitorBatting9PlayerID', 'VisitorBatting9Name', 'HomeBatting1PlayerID', 'HomeBatting1Name', 'HomeBatting2PlayerID', 'HomeBatting2Name', 'HomeBatting3PlayerID', 'HomeBatting3Name', 'HomeBatting4PlayerID', 'HomeBatting4Name', 'HomeBatting5PlayerID', 'HomeBatting5Name', 'HomeBatting6PlayerID', 'HomeBatting6Name', 'HomeBatting7PlayerID', 'HomeBatting7Name', 'HomeBatting8PlayerID', 'HomeBatting8Name', 'HomeBatting9PlayerID', 'HomeBatting9Name']
game_df.drop(col_del,axis=1,inplace=True)
game_df = game_df.rename(columns={"VisitingTeam":"VisitorTeam","VisitingTeamGameNumber":"VisitorTeamGameNumber"})

away_inning = game_df['LengthInOuts']//2
home_inning = game_df['LengthInOuts']- away_inning


game_df['VisitorOffInn'] = away_inning
game_df['VisitorDifInn'] = home_inning
game_df['HomeOffInn'] = home_inning
game_df['HomeDifInn'] = away_inning

game_df['Date'] = pd.to_datetime(game_df['Date'].astype(str), format='%Y%m%d')
game_df['current_year'] = game_df['Date'].dt.year
game_df['Home_team_won?'] = game_df['HomeRunsScore'] > game_df['VisitorRunsScored']
game_df['Visitor_team_won?'] = game_df['HomeRunsScore'] < game_df['VisitorRunsScored']


bat_stat = [ 'TeamGameNumber','Team','AB','H','D','T','HR','RBI','SH','SF',
'HBP','BB','IBB','K','SB','CS','LOB','Pitchers',
'ER','TER','WP','Balks','PO','A','E','Passed','DB','TP','OffInn','DifInn','_team_won?']
pit_stat = ['H', 'D', 'T', 'HR', 'RBI', 'SH', 'SF', 'HBP', 'BB', 'IBB', 'K', 'LOB', 'ER','DB','AB']

select_stat = ['Date']
select_stat += ['Home'+i for i in bat_stat]+['Visitor'+i for i in pit_stat]
rename1 = {'Visitor'+i:'Pit'+i for i in pit_stat}
rename2 = {'Home'+i:i for i in bat_stat}

## Home 시각 
select_stat = ['Date']
select_stat += ['Home'+i for i in bat_stat]+['Visitor'+i for i in pit_stat]
rename1 = {'Visitor'+i:'Pit'+i for i in pit_stat}
rename2 = {'Home'+i:i for i in bat_stat}
Home = game_df.copy()
Home = Home[select_stat].rename(columns=rename1)
Home = Home.rename(columns=rename2)
# away 시각
select_stat = ['Date']
select_stat += ['Visitor'+i for i in bat_stat]+['Home'+i for i in pit_stat]
rename1 = {'Home'+i:'Pit'+i for i in pit_stat}
rename2 = {'Visitor'+i:i for i in bat_stat}
Visitor = game_df.copy()
Visitor = Visitor[select_stat].rename(columns=rename1)
Visitor = Visitor.rename(columns=rename2)

sep_team = pd.concat([Home,Visitor])
sep_team = sep_team.rename(columns={'PitDB':'DP'})
sep_team['year']= sep_team.Date.dt.year
sep_team.sort_values(['year', 'TeamGameNumber'], inplace=True)

In [5]:
col = ['AB', 'H', 'D', 'T', 'HR', 'RBI','DP',
       'SF', 'BB', 'K', 'SB', 'CS', 'LOB', 'Pitchers', 'ER', 'TER', 'WP','PitAB',
       'Balks', 'PO', 'A', 'E', 'Passed', 'DB', 'TP', 'OffInn', 'DifInn',
       'PitH', 'PitD', 'PitT', 'PitHR', 'PitRBI', 'PitSF',
       'PitBB', 'PitK', 'PitLOB', 'PitER']

In [6]:
trend = sep_team.copy()
sep_team[col] = sep_team.groupby(['year','Team'])[col].transform(lambda x: x.expanding(1).sum())
sep_team[col] = sep_team[col].subtract(trend[col])
trend['year']= trend.Date.dt.year
trend[col] = trend.groupby(['year','Team'])[col].transform(lambda x: x.rolling(window=5,closed='left').sum())
sep_team = sep_team[sep_team['TeamGameNumber']>=10]
trend = trend[trend['TeamGameNumber']>=10]


In [7]:

select_feature = ['Date', 'TeamGameNumber', 'Team','_team_won?']
# feature selection
select_feature += ['AB', 'H', 'D', 'T', 'HR', 'RBI','BB', 'K','DP', 'HBP','IBB','SF','SH',#bat stat
                    'ER','OffInn','DifInn' # game info
                    ,'PitAB','PitH', 'PitD', 'PitT', 'PitHR', 'PitRBI', 'PitSF', 'PitBB',"PitIBB","PitHBP" ,'PitK','PitLOB', 'PitER'
                    ]
sep = sep_team[select_feature] 
# H, OBP, SLG, ISO, Babip, RC, wOBA X, K, BB, RBI, K/B, HR   
# 타자 가공
sep['RC'] = ((sep['H']+sep['BB']+sep['HBP']-sep['DP'])
        *(sep['H']+2*sep['D']+3*sep['T']+4*sep['HR']+0.52*(sep['SF']+sep['SH'])+0.26*(sep['BB']+sep['HBP']-sep['IBB']))
        )/(sep['AB']*sep['AB']) #득점 생산


#사구,사사구, 고의사구 통합
sep['BB'] += sep['HBP']+sep['IBB']
sep['PitBB'] += sep['PitHBP']+sep['PitIBB']
#del sep['HBP'],sep['IBB'],sep['PitHBP'],sep['PitIBB']
# 희생번트, 희생플라이 통합
sep['PitSF'] += sep_team['PitSH']
sep['SF'] += sep['SH']
#del sep['SH'],sep['PitSH']

sep['K/B'] = sep['K']/(sep['BB']+0.5)
sep['PA'] = sep['AB'] - sep['BB'] - sep['SF'] # 타수
sep['Babip'] = (sep['H']+sep['D']+sep['T']) / (sep['PA']-sep['K']-sep['HR']-sep['SF']) # 인플레이 타구
sep['SLG'] = (sep['H']+2*sep['D']+3*sep['T']+4*sep['HR'])/sep['PA'] # 장타율
sep['H'] = sep['H']/sep['PA'] # 안타율
sep['D'] = sep['D']/sep['PA'] # 안타율
sep['T'] = sep['T']/sep['PA'] # 안타율
sep['HR'] = sep['HR']/sep['PA'] # 안타율
sep['BB'] = sep['BB']/sep['PA'] # 볼넷
sep['K'] = sep['K']/sep['PA']
sep['RBI'] = sep['RBI']/sep['TeamGameNumber'] # 득점

sep['OBP'] = (sep['H']+sep['BB']) #출루율
sep['OPS'] = sep['SLG']+sep['OBP'] #OPS
sep['GPA'] = (1.8*sep['OBP']+sep['SLG'])/4#GPA(Gross Production Average) - park factor를 적용해야함
sep['ISO'] = sep['SLG'] - sep['H']


# 투수 가공
# PitH, PitOBP, PitSLG, PitOPS, PitHR, EAR, PitB, PitK, PitK/B, WHIP, kwERA
sep['ERA'] = sep['PitER']/sep['DifInn']
sep['TotalH'] = sep['PitH']+sep['PitD']+sep['PitT']+sep['PitHR']
sep['TotalBB'] = sep['PitBB']+sep['PitHBP']
sep['PitPA'] = sep['PitAB'] - sep['TotalBB'] - sep['PitSF']
sep['kwERA'] = (5.40-12*(sep['PitK'] - sep['TotalBB']))/sep['PitPA']
sep['WHIP'] = (sep['TotalH']+sep['TotalBB'])/sep['DifInn']
sep['PitK'] = sep['PitK']/sep['DifInn']
sep['PitB'] = sep['TotalBB']/sep['DifInn']
sep['PitK/B'] = sep['PitK']/(sep['PitB']+0.5)

sep['PitSLG'] = (sep['PitH']+2*sep['PitD']+3*sep['PitT']+4*sep['PitHR'])/sep['PitPA'] # 피장타율
sep['PitH'] = sep['PitH']/sep['PitPA'] # 피안타율
sep['PitOBP'] = (sep['TotalH']+sep['TotalBB']) / sep['PitAB'] #피출루율
sep['PitOPS'] = sep['PitSLG']+sep['PitOBP'] #피OPS
sep['PitHR'] /= sep['PitPA']

#temp = sep[['Date', 'TeamGameNumber', 'Team', '_team_won?','GPA', 'OPS','OBP','H','SLG','Babip','RC']]
temp = sep
game_log = game_df[['Date','VisitorTeam','HomeTeam','VisitorTeamGameNumber','HomeTeamGameNumber', 'VisitorStartingPitcherName', 'HomeStartingPitcherName']]
game_log = pd.merge(game_log,temp,left_on = ['Date','VisitorTeam'], right_on = ['Date','Team'],how='left')
game_log = pd.merge(game_log,temp,left_on = ['Date','HomeTeam'], right_on = ['Date','Team'],how='left')

select_feature1 = ['Date', 'TeamGameNumber', 'Team','_team_won?']
select_feature2 = ['AB', 'H', 'D', 'T', 'HR', 'RBI','BB', 'K','DP', 'HBP','IBB','SF','SH',#bat stat
                    'ER','OffInn','DifInn' # game info
                    ,'PitAB','PitH', 'PitD', 'PitT', 'PitHR', 'PitRBI', 'PitSF', 'PitBB',"PitIBB","PitHBP" ,'PitK','PitLOB', 'PitER'
                    ]
sep = trend[select_feature1 + select_feature2]

# H, OBP, SLG, ISO, Babip, RC, wOBA X, K, BB, RBI, K/B, HR   
# 타자 가공
sep['RC'] = ((sep['H']+sep['BB']+sep['HBP']-sep['DP'])
        *(sep['H']+2*sep['D']+3*sep['T']+4*sep['HR']+0.52*(sep['SF']+sep['SH'])+0.26*(sep['BB']+sep['HBP']-sep['IBB']))
        )/(sep['AB']*sep['AB']) #득점 생산


#사구,사사구, 고의사구 통합
sep['BB'] += sep['HBP']+sep['IBB']
sep['PitBB'] += sep['PitHBP']+sep['PitIBB']
#del sep['HBP'],sep['IBB'],sep['PitHBP'],sep['PitIBB']
# 희생번트, 희생플라이 통합
sep['PitSF'] += sep_team['PitSH']
sep['SF'] += sep['SH']
#del sep['SH'],sep['PitSH']

sep['K/B'] = sep['K']/(sep['BB']+0.5)
sep['PA'] = sep['AB'] - sep['BB'] - sep['SF'] # 타수
sep['Babip'] = (sep['H']+sep['D']+sep['T']) / (sep['PA']-sep['K']-sep['HR']-sep['SF']) # 인플레이 타구
sep['SLG'] = (sep['H']+2*sep['D']+3*sep['T']+4*sep['HR'])/sep['PA'] # 장타율
sep['H'] = sep['H']/sep['PA'] # 안타율
sep['D'] = sep['D']/sep['PA'] # 안타율
sep['T'] = sep['T']/sep['PA'] # 안타율
sep['HR'] = sep['HR']/sep['PA'] # 안타율
sep['BB'] = sep['BB']/sep['PA'] # 볼넷
sep['RBI'] = sep['RBI']/5 # 득점
sep['K'] = sep['K']/sep['PA']

sep['OBP'] = (sep['H']+sep['BB']) #출루율
sep['OPS'] = sep['SLG']+sep['OBP'] #OPS
sep['GPA'] = (1.8*sep['OBP']+sep['SLG'])/4#GPA(Gross Production Average) - park factor를 적용해야함
sep['ISO'] = sep['SLG'] - sep['H']

# 투수 가공
# PitH, PitOBP, PitSLG, PitOPS, PitHR, EAR, PitB, PitK, PitK/B, WHIP, kwERA
sep['ERA'] = sep['PitER']/sep['DifInn']
sep['TotalH'] = sep['PitH']+sep['PitD']+sep['PitT']+sep['PitHR']
sep['TotalBB'] = sep['PitBB']+sep['PitHBP']
sep['PitPA'] = sep['PitAB'] - sep['TotalBB'] - sep['PitSF']
sep['kwERA'] = (5.40-12*(sep['PitK'] - sep['TotalBB']))/sep['PitPA']
sep['WHIP'] = (sep['TotalH']+sep['TotalBB'])/sep['DifInn']
sep['PitK'] = sep['PitK']/sep['DifInn']
sep['PitB'] = sep['TotalBB']/sep['DifInn']
sep['PitK/B'] = sep['PitK']/(sep['PitB']+0.5)


sep['PitSLG'] = (sep['PitH']+2*sep['PitD']+3*sep['PitT']+4*sep['PitHR'])/sep['PitPA'] # 피장타율
sep['PitH'] = sep['PitH']/sep['PitPA'] # 피안타율
sep['PitOBP'] = (sep['TotalH']+sep['TotalBB']) / sep['PitAB'] #피출루율
sep['PitOPS'] = sep['PitSLG']+sep['PitOBP'] #피OPS
sep['PitHR'] /= sep['PitPA']

#temp = sep[['Date', 'TeamGameNumber', 'Team', '_team_won?','GPA', 'OPS','OBP','H','SLG','Babip','RC']]
select = ['H','OBP','SLG','ISO','Babip','RC','K','BB','RBI','K/B','HR','PitH','PitOBP','PitSLG','PitOPS','PitHR','ERA','PitB','PitK','PitK/B','WHIP']
temp = sep[select_feature1 + select]

game_log = game_log[game_log.VisitorTeamGameNumber>=10]
game_log = game_log[game_log.HomeTeamGameNumber>=10]

trend_columns = select_feature1 + ["trend" + i for i in select]
temp.columns = trend_columns
temp.drop(['TeamGameNumber', '_team_won?'],axis=1, inplace=True)
game_log = pd.merge(game_log,temp,left_on = ['Date','HomeTeam'], right_on = ['Date','Team'],how='left')
game_log = pd.merge(game_log,temp,left_on = ['Date','VisitorTeam'], right_on = ['Date','Team'],how='left')



In [8]:
!pip install unidecode

In [9]:
from unidecode import unidecode

In [10]:
def remove_aster(s):
  r = s.lower()
  if s[-1] == '*':
    r = r[:-1]
  tmp = r.split()
  if len(tmp) != 2:
    if tmp[2] == 'iii':
      r = ' '.join(tmp[:2])
  return r

In [11]:
game_log.HomeStartingPitcherName = game_log.HomeStartingPitcherName.transform(lambda x: remove_aster(x))
game_log.VisitorStartingPitcherName = game_log.VisitorStartingPitcherName.transform(lambda x: remove_aster(x))
game_log.replace({'guillermo moscoso (corredor)':'guillermo moscoso','wade matthew leblanc':'wade leblanc'}, inplace=True)
all_pitchers = np.union1d(game_log.HomeStartingPitcherName.unique(), game_log.VisitorStartingPitcherName.unique())

In [12]:
names_to_chage = ['Charles Leesman*', 'Dave Bush', 'Duane Underwood Jr.', 'Roberto Hernandez','Hyun Jin Ryu*', 'JC Ramirez', 'Jake Faria', 'Jakob Junis', 'Josh A. Smith', 'Lance McCullers Jr.', 'Luke French*', 'Mark Leiter Jr.', 'Matthew Boyd*', 'Matthew Festa', 'Mike Fiers','Mike Wright Jr.', 'Nate Karns', 'Sam Deduno', 'T.J. House*','Tommy Milone*', 'Vince Velasquez', 'Wade LeBlanc*','Yunesky Maya' ]
changed_name= ['Charlie Leesman', 'David Bush', 'Duane Underwood','Fausto Carmona', 'Hyun-Jin Ryu', 'J.C. Ramirez', 'Jacob Faria','Jake Junis', 'Josh Smith', 'Lance Mccullers', 'Lucas French','Mark Leiter', 'Matt Boyd', 'Matt Festa', 'Michael Fiers','Mike Wright', 'Nathan Karns', 'Samuel Deduno', 'TJ House','Tom Milone', 'Vincent Velasquez', 'Wade LeBlanc','Yuniesky Maya']
new_names = { name:new_name for name, new_name in zip(names_to_chage, changed_name) }

In [13]:
root_dir = './data/Retrosheet_2010_2019/'

pitcher_df = pd.DataFrame()
for i in range(2009, 2019):
  df = pd.read_csv(root_dir+str(i)+'.csv')
  df = df.drop(['Rk','Age', 'Lg', 'SHO'], axis=1)
  df['year'] = i + 1
  df.drop(df.index[-1], axis=0, inplace=True)
  duplicated_names = df[df.Tm == 'TOT'].Name
  df.drop(df[(df.Tm == 'TOT')].index, inplace=True)
  pitcher_df = pd.concat([pitcher_df, df], ignore_index=True)
pitcher_df.Name = pitcher_df.Name.str.split('\\').str[0]
pitcher_df.replace(new_names, inplace=True)
pitcher_df.Name = pitcher_df.Name.transform(lambda x: unidecode(x))
pitcher_df.Name = pitcher_df.Name.transform(lambda x: remove_aster(x))
pitcher_df = pitcher_df[pitcher_df.Name.isin(all_pitchers)]

In [14]:
team_pitching = pd.DataFrame()
for i in range(2009, 2019):
  df = pd.read_csv(root_dir+'team_' + str(i) + '.csv')
  df = df.drop(['#P', 'PAge', 'RA/G', 'LOB', 'cSho', 'tSho'], axis=1)
  df['year'] = i + 1
  team_pitching = pd.concat([team_pitching, df], ignore_index=True)
team_pitching.replace({'Los Angeles Angels of Anaheim':'Los Angeles Angels'}, inplace = True)

In [15]:
game_log.replace({"MIA":"FLO"}, inplace=True)
pitcher_df.replace({"MIA":"FLA"}, inplace=True)

Team Name Handling

In [16]:
team_name_a = """
ARI
ATL
BAL
BOS
CHN
CHA
CIN
CLE
COL
DET
FLO
HOU
KCA
ANA
LAN
MIL
MIN
NYN
NYA
OAK
PHI
PIT
SDN
SEA
SFN
SLN
TBA
TEX
TOR
WAS
"""

In [17]:
team_name_b = """
ARI
ATL
BAL
BOS
CHC
CHW
CIN
CLE
COL
DET
FLA
HOU
KCR
LAA
LAD
MIL
MIN
NYM
NYY
OAK
PHI
PIT
SDP
SEA
SFG
STL
TBR
TEX
TOR
WSN
"""

In [18]:
team_name_c = """
Arizona Diamondbacks
Atlanta Braves 
Baltimore Orioles
Boston Red Sox
Chicago Cubs
Chicago White Sox
Cincinnati Reds
Cleveland Indians
Colorado Rockies
Detroit Tigers
Florida Marlins
Houston Astros
Kansas City Royals
Los Angeles Angels
Los Angeles Dodgers
Milwaukee Brewers
Minnesota Twins
New York Mets
New York Yankees
Oakland Athletics
Philadelphia Phillies
Pittsburgh Pirates
San Diego Padres
Seattle Mariners
San Francisco Giants
St. Louis Cardinals
Tampa Bay Rays
Texas Rangers
Toronto Blue Jays
Washington Nationals
"""

In [19]:
TeamName1 = {name:new_name for name, new_name in zip(team_name_b.split('\n'), team_name_a.split('\n')) }
TeamName2 = {name:new_name for name, new_name in zip(team_name_c.split('\n'), team_name_a.split('\n')) }
pitcher_df.replace(TeamName1, inplace=True)
team_pitching.replace(TeamName2, inplace=True)
pitcher_df.columns = ['StartPitcher'+ i for i in pitcher_df.columns]
team_pitching.columns = ['StartPitcher'+ i for i in team_pitching.columns]

In [21]:
game_log['year'] = game_log.Date.dt.year
new_df = game_log[['year', 'HomeStartingPitcherName', 'HomeTeam']]
new_df.columns = ['year', 'Name', 'Team']
tmp = game_log[['year', 'VisitorStartingPitcherName', 'VisitorTeam']]
tmp.columns = ['year', 'Name', 'Team']
new_df = pd.concat([new_df,tmp])
new_df.drop_duplicates(inplace=True)

In [22]:
new_df = pd.merge(new_df,pitcher_df,left_on = ['year','Name', 'Team'], right_on = ['StartPitcheryear','StartPitcherName','StartPitcherTm'], how='left')
# new_df2 = pd.merge(new_df,team_pitching,left_on = ['year', 'Team'], right_on = ['StartPitcheryear','StartPitcherTm'],how='left')

In [23]:
new_df.iloc[2].Name , pitcher_df.iloc[181].StartPitcherName

('dana eveland', 'jeff niemann')

In [248]:

game_log2 = pd.merge(game_log,pitcher_df,left_on = ['year','HomeStartingPitcherName', 'HomeTeam'], right_on = ['StartPitcheryear','StartPitcherName','StartPitcherTm'],how='left')
game_log2 = pd.merge(game_log2,pitcher_df,left_on = ['year','VisitorStartingPitcherName', 'VisitorTeam'], right_on = ['StartPitcheryear','StartPitcherName','StartPitcherTm'],how='left')


In [32]:
pitcher_df.to_csv('pitcher_2010_to_2019.csv', mode='w')
team_pitching.to_csv('teampitcher_2010_to_2019.csv', mode='w')